In [1]:
import pandas as pd

In [2]:
submit_to_agent = pd.read_pickle('./data/submit_to_agent.p') 

In [3]:
submit_to_agent

,_id,userId,full_name,datetime,reason,message
0,5e747a3b88c9e151b2f1f29b,3432603620089385,มงคล ชุ่มใจ,2020-03-20T08:09:31.466143,not_understand,เบี้ย9999บ.คุ้มครอง​อะไรบ้าง
1,5e747a445bc258ca00eabfc7,2739695839400025,สุนทร สีหา,2020-03-20T08:09:40.274293,$ensitive,อยากทราบว่า.ทางรัฐบาลให้พักชำระหนี้ เงินติดล้อ...
2,5e747b3d88c9e151b2f1f2e7,2317879594928073,Chayatorn Arunchanachai,2020-03-20T08:13:49.144198,not_understand,โคโรนา
3,5e747c2d88c9e151b2f1f309,3654936474576279,Nampol Sawitree,2020-03-20T08:17:49.969109,not_understand,092-2748189
4,5e747d4888c9e151b2f1f33a,4065967033417170,ราชันย์ จันทะแค,2020-03-20T08:22:32.396304,$ensitive,กู้กับเงินล้อนิละครับ
...,...,...,...,...,...,...
12351,5ec4e07bac4f5c1d305fe65c,3100678436658802,หนุ่มอนุบาล สุดหล่อ,2020-05-20T07:47:07.656826,not_response,https://scontent.xx.fbcdn.net/v/t1.15752-9/968...
12352,5ec4e111ac4f5c1d305fe65d,3126007527457002,นุชรี ทองโอ,2020-05-20T07:49:37.617887,not_response,
12353,5ec4e16bac4f5c1d305fe66e,4749158108443717,Meree Meree,2020-05-20T07:51:07.027190,not_response,{{DLJingjai}}
12354,5ec4e189ac4f5c1d305fe670,2939113419516575,ตีบ แดนไต้,2020-05-20T07:51:37.688190,not_response,


In [4]:
def to_dategooglesheet(date):
	return date.strftime('%Y%m%d')

In [5]:
def to_datestr(date):
	return date.strftime('%Y-%m-%d')

In [6]:
from datetime import datetime, timedelta, date
# Format time to match with the data
today = date.today()
td = to_datestr(today)

In [7]:
def get_date(text):
    date = text[:10]
    return date

In [24]:
def get_no_not_understand(submit_to_agent, date:str=td):
    submit_to_agent['date'] = submit_to_agent.datetime.apply(lambda x: get_date(x))
    df = submit_to_agent[submit_to_agent['date'] == date]
    df = df.loc[df.reason=='not_understand'].reset_index()
    return len(df)

In [9]:
length = get_no_not_understand(submit_to_agent, date='2020-03-20')

In [10]:
print(length)

83


In [51]:
fbr_df = pd.DataFrame(columns=['date', 'no. of not_understand'])

In [11]:
fbr_df.head()

,date,no. of not_understand


In [52]:
def update_fbr_df(submit_to_agent, date:str=td, fbr_df=fbr_df):
    submit_to_agent['date'] = submit_to_agent.datetime.apply(lambda x: get_date(x))
    df = submit_to_agent[submit_to_agent['date'] == date]
    df = df.loc[df.reason=='not_understand'].reset_index()
    not_understand = len(df)
    date_obj = datetime.strptime(date, '%Y-%m-%d')
    df2 = pd.DataFrame({"date": [to_dategooglesheet(date_obj)], 
                    "no. of not_understand":[not_understand]})
    return fbr_df.append(df2)

In [34]:
update_fbr_df(submit_to_agent, '2020-03-20', fbr_df)

,date,no. of not_understand
0,20200320,83


In [53]:
original_date = '2020-03-20'

for i in range(62):
    dt = datetime.strptime(original_date, '%Y-%m-%d')
    dt += timedelta(days=i)
    date = dt.strftime('%Y-%m-%d')
    fbr_df = update_fbr_df(submit_to_agent, date, fbr_df)

In [54]:
fbr_df

,date,no. of not_understand
0,20200320,83
0,20200321,99
0,20200322,112
0,20200323,121
0,20200324,87
...,...,...
0,20200516,43
0,20200517,44
0,20200518,67
0,20200519,85


In [48]:
fbr_df = fbr_df.reset_index(inplace=True)
fbr_df_final = fbr_df['date', 'no. of not_understand']

AttributeError: 'NoneType' object has no attribute 'reset_index'

In [15]:
latest_index = len(submit_to_agent) - 1
print(latest_index)

12355


In [55]:
fbr_df.to_excel('./excel/fbr_not_understand.xlsx')